In [1]:
!pip install ultralytics
!pip install -U albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 8.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.20
    Uninstalling albucore-0.0.20:
      Successfully uninstalled albucore-0.0.20
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.21
    Uninstalling albumentations-1.4.21:
      Successfully uninstalled albumentations-1.4.21


In [ ]:
"""
import os
import shutil
import pandas as pd

# Define paths
csv_file_path = '/kaggle/input/datathon-ai-qualification-round/train_data.csv'
source_folder = '/kaggle/input/datathon-ai-qualification-round/train/train'
destination_base =  '/kaggle/working/Sehirler/train/train'

# Load CSV data
train_data = pd.read_csv(csv_file_path)

# Create city directories in the destination folder
for city in train_data['city'].unique():
    city_folder = os.path.join(destination_base, city)
    os.makedirs(city_folder, exist_ok=True)

# Move files to their respective city folders
for index, row in train_data.iterrows():
    filename = row['filename']
    city = row['city']
    
    source_file = os.path.join(source_folder, filename)
    destination_file = os.path.join(destination_base, city, filename)

    # Check if the source file exists
    if os.path.exists(source_file):
        shutil.copy(source_file, destination_file)
    else:
        print(f"Warning: {source_file} does not exist.")
"""

In [2]:
import os
import shutil
import pandas as pd

# CSV dosyasını yükle
csv_path = "/kaggle/input/datathon-ai-qualification-round/train_data.csv"  # CSV dosyanızın tam yolu
image_folder = "/kaggle/input/datathon-ai-qualification-round/train/train"  # Fotoğrafların bulunduğu klasör
output_folder = "/kaggle/working/Sehirler"  # Çıktı klasörü

# CSV'yi oku
df = pd.read_csv(csv_path)

# Şehir klasörlerini oluştur ve fotoğrafları taşı
for _, row in df.iterrows():
    city = row['city']
    filename = row['filename']
    
    # Şehir klasörünün yolunu oluştur
    city_folder = os.path.join(output_folder, city)
    os.makedirs(city_folder, exist_ok=True)
    
    # Fotoğrafın mevcut ve hedef yolunu oluştur
    source_path = os.path.join(image_folder, filename)
    target_path = os.path.join(city_folder, filename)
    
    # Fotoğrafı taşı
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)
    else:
        print(f"{filename} bulunamadı.")

print("Fotoğraflar başarıyla taşındı.")


Fotoğraflar başarıyla taşındı.


In [3]:
# Şehirleri sınıflara çevir
city_to_class = {city: idx for idx, city in enumerate(df['city'].unique())}

# YOLO formatında etiket dosyalarını oluştur
for _, row in df.iterrows():
    city = row['city']
    filename = row['filename']
    image_path = os.path.join(output_folder, city, filename)
    
    if os.path.exists(image_path):
        # Etiket dosyasının yolunu oluştur
        label_path = image_path.replace(".jpg", ".txt")
        class_id = city_to_class[city]
        
        # Tüm görüntüyü kapsayan bir "box" kullanıyoruz
        with open(label_path, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")  # x_center, y_center, width, height


In [4]:
city_to_class

{'Istanbul': 0, 'Ankara': 1, 'Izmir': 2}

## BURAYA KADAR HER ŞEY AYNI

In [ ]:
import os
import shutil
import random

# Eğitim, doğrulama ve test setleri için oranlar
train_ratio = 0.7
val_ratio = 0.2

# Tüm verileri shuffle'la
images = [os.path.join(root, file) 
          for root, _, files in os.walk(output_folder) for file in files if file.endswith(".jpg")]
random.shuffle(images)

# Verileri böl
train_split = int(len(images) * train_ratio)
val_split = int(len(images) * (train_ratio + val_ratio))

train_files = images[:train_split]
val_files = images[train_split:val_split]
test_files = images[val_split:]

# Eğitim, doğrulama ve test klasörlerini oluştur
dataset_folder = "/kaggle/working/SehirlerDataset"
os.makedirs(os.path.join(dataset_folder, "train"), exist_ok=True)
os.makedirs(os.path.join(dataset_folder, "val"), exist_ok=True)
os.makedirs(os.path.join(dataset_folder, "test"), exist_ok=True)

# Dosyaları ve ilgili .txt dosyalarını taşı
for file_set, folder in zip([train_files, val_files, test_files], ["train", "val", "test"]):
    for file in file_set:
        # .jpg dosyasını taşı
        target_image_path = os.path.join(dataset_folder, folder, os.path.basename(file))
        shutil.move(file, target_image_path)

        # İlgili .txt dosyasını kontrol et ve taşı
        label_file = file.replace(".jpg", ".txt")
        if os.path.exists(label_file):
            target_label_path = os.path.join(dataset_folder, folder, os.path.basename(label_file))
            shutil.move(label_file, target_label_path)

print("Resimler ve etiketler başarıyla bölündü ve taşındı.")


In [ ]:
import yaml

# YAML dosyası için veri
data_yaml = {
    'train': '/kaggle/working/SehirlerDataset/train',
    'val': '/kaggle/working/SehirlerDataset/val',
    'nc': 3,
    'names': ['Istanbul', 'Ankara', 'Izmir']
}

# YAML dosyasını oluştur
yaml_path = "/kaggle/working/SehirlerDataset/data.yaml"
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(data_yaml, yaml_file, default_flow_style=False)



In [8]:
import os

def list_unique_extensions(folder_path):
    """
    Belirtilen bir klasördeki eşsiz dosya uzantılarını listeleyen fonksiyon.

    Args:
        folder_path (str): Klasör yolu.

    Returns:
        set: Bulunan benzersiz uzantıları içeren set.
    """
    unique_extensions = set()
    
    # Klasördeki dosyaları gez
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            _, ext = os.path.splitext(file)
            if ext:  # Eğer dosya uzantısı varsa
                unique_extensions.add(ext)

    return unique_extensions

# Kullanım örneği
folder_path = "/kaggle/working/SehirlerDatasetSplit"  # Klasör yolunu düzenleyin

extensions = list_unique_extensions(folder_path)

if extensions:
    print("Klasördeki benzersiz dosya uzantıları:")
    for ext in extensions:
        print(ext)
else:
    print("Klasörde herhangi bir dosya uzantısı bulunamadı.")


Klasördeki benzersiz dosya uzantıları:
.yaml
.txt
.jpg


In [ ]:
from ultralytics import YOLO

# Load a pretrained model
model = YOLO("yolo11n.pt")

# Train the model on your custom dataset
model.train(data="/kaggle/working/SehirlerDataset/data.yaml", epochs=100, imgsz=640, device=[0, 1], batch=-1)

In [5]:
import os
import shutil
import random

# Eğitim ve doğrulama setleri için oranlar
train_ratio = 0.94  # Eğitim oranı (%80)
val_ratio = 0.06    # Doğrulama oranı (%20)

# Veri seti klasörleri
output_folder = "/kaggle/working/Sehirler"  # Ana klasör (Ankara, Izmir, Istanbul)
dataset_folder = "/kaggle/working/SehirlerDatasetSplit"  # Bölünecek klasör

# Eğitim ve doğrulama klasörlerini oluştur
os.makedirs(os.path.join(dataset_folder, "train"), exist_ok=True)
os.makedirs(os.path.join(dataset_folder, "val"), exist_ok=True)

# Alt klasörleri dolaş
for class_name in os.listdir(output_folder):
    class_path = os.path.join(output_folder, class_name)
    if not os.path.isdir(class_path):
        continue  # Sadece klasörlere odaklan

    # Sınıfa ait tüm resim dosyalarını al
    images = [os.path.join(class_path, file) for file in os.listdir(class_path) if file.endswith(".jpg")]
    random.shuffle(images)  # Rastgele karıştır

    # Eğitim ve doğrulama setlerini ayır
    train_split = int(len(images) * train_ratio)
    train_files = images[:train_split]
    val_files = images[train_split:]

    # Eğitim ve doğrulama setlerine taşı
    for file_set, folder in zip([train_files, val_files], ["train", "val"]):
        target_folder = os.path.join(dataset_folder, folder)  # Klasör isimlerini düz tut
        for file in file_set:
            # .jpg dosyasını taşı
            target_image_path = os.path.join(target_folder, os.path.basename(file))
            shutil.move(file, target_image_path)

            # İlgili .txt dosyasını kontrol et ve taşı
            label_file = file.replace(".jpg", ".txt")
            if os.path.exists(label_file):
                target_label_path = os.path.join(target_folder, os.path.basename(label_file))
                shutil.move(label_file, target_label_path)

print("Resimler ve etiketler başarıyla eğitim ve doğrulama setlerine bölündü ve taşındı.")


Resimler ve etiketler başarıyla eğitim ve doğrulama setlerine bölündü ve taşındı.


In [6]:
import yaml

# YAML dosyası için veri
data_yaml = {
    'train': '/kaggle/working/SehirlerDatasetSplit/train',
    'val': '/kaggle/working/SehirlerDatasetSplit/val',
    'nc': 3,
    'names': ['Istanbul', 'Ankara', 'Izmir']
}

# YAML dosyasını oluştur
yaml_path = "/kaggle/working/SehirlerDatasetSplit/data.yaml"
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(data_yaml, yaml_file, default_flow_style=False)



In [ ]:
from ultralytics import YOLO

# Load a pretrained model
model = YOLO("yolo11m.pt")

# Train the model on your custom dataset
model.train(data="/kaggle/working/SehirlerDatasetSplit/data.yaml", epochs=100, imgsz=640, device=[0, 1])

In [ ]:
!ls /kaggle/working/SehirlerDatasetSplit/train -1 | wc -l

# SADECE LİNEAR DÜZ CLIP

In [1]:
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-e09sjqyg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-e09sjqyg
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=dec83c865ffd260090483557c4937614c4350c22ceb5cf0676bf43c1b3e560c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-jw8b1h34/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [3]:
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
import clip
# Eğer openclip kullanacaksanız:
# !pip install open_clip_torch
# import open_clip

# MODEL YÜKLEME (OpenAI CLIP)
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device)  # ya da daha büyük bir model
model.eval()

# VERİYİ YÜKLE
df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/train_data.csv")  # filename, city kolonları var
image_paths = df['filename'].apply(lambda x: f"/kaggle/input/datathon-ai-qualification-round/train/train/{x}")
cities = df['city'].unique().tolist()

# TEXT PROMPTLARINI HAZIRLA
text_prompts = [f"A photo of Türkiye in {city}." for city in cities]

# TEXT EMBEDDINGS
with torch.no_grad():
    text_tokens = clip.tokenize(text_prompts).to(device)
    text_features = model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

# ZERO-SHOT SINIFLANDIRMA TESTİ
correct = 0
for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_path = row['filename']
    label = row['city']
    image = preprocess(Image.open(f"/kaggle/input/datathon-ai-qualification-round/train/train/{img_path}")).unsqueeze(0).to(device)
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    
    similarity = (image_features @ text_features.T).squeeze(0)
    # En yüksek skor hangi şehir ise onu tahmin et
    pred_idx = similarity.argmax().item()
    pred_city = cities[pred_idx]

    if pred_city == label:
        correct += 1

accuracy = correct / len(df)
print("Zero-Shot Accuracy:", accuracy)

100%|████████████████████████████████████████| 891M/891M [00:05<00:00, 157MiB/s]
100%|██████████| 7000/7000 [04:40<00:00, 24.93it/s]

Zero-Shot Accuracy: 0.7591428571428571


In [ ]:
import torch
from PIL import Image
import pandas as pd
from tqdm import tqdm
import clip
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import f1_score

# -----------------
# Hyperparametreler
# -----------------
batch_size = 32
learning_rate = 1e-3
epochs = 500

device = "cuda" if torch.cuda.is_available() else "cpu"

# ---------
# MODEL YÜKLE
# ---------
model, preprocess = clip.load("ViT-L/14", device=device)
model.eval()

# -----------------
# DATASET HAZIRLAMA
# -----------------
df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/train_data.csv")  # filename, city kolonları var
image_paths = df['filename'].apply(lambda x: f"/kaggle/input/datathon-ai-qualification-round/train/train/{x}").values
cities = df['city'].unique().tolist()
city_to_idx = {c: i for i, c in enumerate(cities)}
labels = df['city'].apply(lambda x: city_to_idx[x]).values

class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, label

dataset = ImageDataset(image_paths, labels, preprocess)

# Basitçe tüm verileri tek seferde embedding'e çevirebiliriz (veri küçükse).
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

all_image_features = []
all_labels = []

with torch.no_grad():
    for imgs, labs in tqdm(dataloader):
        imgs = imgs.to(device)
        image_features = model.encode_image(imgs)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        all_image_features.append(image_features.cpu())
        all_labels.append(labs)

all_image_features = torch.cat(all_image_features, dim=0)
all_labels = torch.cat(all_labels, dim=0)

# Veriyi train/val/test olarak ayırın.
# Burada basitçe %80-%20 bölelim örnek olması için.
num_samples = len(all_image_features)
train_size = int(0.6 * num_samples)
val_size = num_samples - train_size

train_features = all_image_features[:train_size]
train_labels = all_labels[:train_size]

val_features = all_image_features[train_size:]
val_labels = all_labels[train_size:]

# ------------------------
# LINEAR PROBING MODELİ
# ------------------------
# CLIP feature boyutunu öğrenelim
feature_dim = train_features.shape[1]
num_classes = len(cities)

linear_classifier = nn.Linear(feature_dim, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_classifier.parameters(), lr=learning_rate)

# ------------------------
# EĞİTİM DÖNGÜSÜ
# ------------------------
for epoch in range(epochs):
    linear_classifier.train()
    # Veriyi random karıştırmak için tekrar dataloader yapabiliriz ama burada direkt batch loop yapalım.
    permutation = torch.randperm(train_size)
    train_features_shuf = train_features[permutation]
    train_labels_shuf = train_labels[permutation]

    for i in range(0, train_size, batch_size):
        batch_feats = train_features_shuf[i:i+batch_size].to(device)
        batch_labs = train_labels_shuf[i:i+batch_size].to(device)

        optimizer.zero_grad()
        outputs = linear_classifier(batch_feats.float())
        loss = criterion(outputs, batch_labs)
        loss.backward()
        optimizer.step()

    # Validation
    linear_classifier.eval()
    with torch.no_grad():
        val_outputs = linear_classifier(val_features.float().to(device))
        val_pred = val_outputs.argmax(dim=1).cpu()  # Tahminler CPU'ya alınır
        # val_labels zaten CPU'da ise direkt kullanabilirsiniz, değilse val_labels.cpu() yapın
        f1 = f1_score(val_labels.cpu(), val_pred, average='macro')
    
    print(f"Validation Macro F1: {f1:.4f}")

# Eğitim bittiğinde en iyi val acc elde ettiğiniz modeli kaydedebilirsiniz.
# test setiniz varsa onu da benzer şekilde değerlendirebilirsiniz.


In [23]:
import torch
from PIL import Image
import pandas as pd
from tqdm import tqdm
import clip
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import f1_score
import os

# ---------------
# Hyperparametreler
# ---------------
batch_size = 32
learning_rate = 1e-3
epochs = 500

device = "cuda" if torch.cuda.is_available() else "cpu"

# ---------
# MODEL YÜKLEME (CLIP)
# ---------
model, preprocess = clip.load("ViT-L/14", device=device)
model.eval()

# -----------------
# TRAIN DATA YÜKLEME
# -----------------
df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/train_data.csv")  # filename, city kolonları var
image_paths = df['filename'].apply(lambda x: f"/kaggle/input/datathon-ai-qualification-round/train/train/{x}").values
cities = df['city'].unique().tolist()
city_to_idx = {c: i for i, c in enumerate(cities)}
labels = df['city'].apply(lambda x: city_to_idx[x]).values

class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, label

dataset = ImageDataset(image_paths, labels, preprocess)

# Tüm verilerden embedding çıkar
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

all_image_features = []
all_labels = []

with torch.no_grad():
    for imgs, labs in tqdm(dataloader):
        imgs = imgs.to(device)
        image_features = model.encode_image(imgs)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        all_image_features.append(image_features.cpu())
        all_labels.append(labs)

all_image_features = torch.cat(all_image_features, dim=0)
all_labels = torch.cat(all_labels, dim=0)

# Veriyi train/val olarak ayıralım (ör: %80-%20)
num_samples = len(all_image_features)
train_size = int(0.8 * num_samples)
val_size = num_samples - train_size

train_features = all_image_features[:train_size]
train_labels = all_labels[:train_size]

val_features = all_image_features[train_size:]
val_labels = all_labels[train_size:]

feature_dim = train_features.shape[1]
num_classes = len(cities)

linear_classifier = nn.Linear(feature_dim, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_classifier.parameters(), lr=learning_rate)

best_f1 = 0.0
model_path = "/kaggle/working/linear_classifier_best.pt"

for epoch in range(epochs):
    linear_classifier.train()
    permutation = torch.randperm(train_size)
    train_features_shuf = train_features[permutation]
    train_labels_shuf = train_labels[permutation]

    # Mini-batch eğitim
    for i in range(0, train_size, batch_size):
        batch_feats = train_features_shuf[i:i+batch_size].to(device)
        batch_labs = train_labels_shuf[i:i+batch_size].to(device)

        optimizer.zero_grad()
        outputs = linear_classifier(batch_feats.float())
        loss = criterion(outputs, batch_labs)
        loss.backward()
        optimizer.step()

    # Validation aşaması
    linear_classifier.eval()
    with torch.no_grad():
        val_outputs = linear_classifier(val_features.float().to(device))
        val_pred = val_outputs.argmax(dim=1).cpu()
        f1 = f1_score(val_labels.cpu(), val_pred, average='macro')
    
    print(f"Epoch {epoch+1}/{epochs} - Validation Macro F1: {f1:.4f}")

    # En iyi model kontrolü
    if f1 > best_f1:
        best_f1 = f1
        torch.save(linear_classifier.state_dict(), model_path)
        print("Best model saved with F1:", best_f1)






Epoch 473/500 - Validation Macro F1: 0.9256
Epoch 474/500 - Validation Macro F1: 0.9249
Epoch 475/500 - Validation Macro F1: 0.9264
Epoch 476/500 - Validation Macro F1: 0.9249
Epoch 477/500 - Validation Macro F1: 0.9249
Epoch 478/500 - Validation Macro F1: 0.9257
Epoch 479/500 - Validation Macro F1: 0.9257
Epoch 480/500 - Validation Macro F1: 0.9257
Epoch 481/500 - Validation Macro F1: 0.9278
Epoch 482/500 - Validation Macro F1: 0.9257
Epoch 483/500 - Validation Macro F1: 0.9249
Epoch 484/500 - Validation Macro F1: 0.9249
Epoch 485/500 - Validation Macro F1: 0.9264
Epoch 486/500 - Validation Macro F1: 0.9249
Epoch 487/500 - Validation Macro F1: 0.9250
Epoch 488/500 - Validation Macro F1: 0.9249
Epoch 489/500 - Validation Macro F1: 0.9250
Epoch 490/500 - Validation Macro F1: 0.9271
Epoch 491/500 - Validation Macro F1: 0.9250
Epoch 492/500 - Validation Macro F1: 0.9264
Epoch 493/500 - Validation Macro F1: 0.9249
Epoch 494/500 - Validation Macro F1: 0.9264
Epoch 495/500 - Validation Macro

/tmp/ipykernel_23/1277279265.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  linear_classifier.load_state_dict(torch.load(model_path))


Linear(in_features=768, out_features=3, bias=True)

In [24]:
# En iyi modeli yükleyelim
linear_classifier.load_state_dict(torch.load(model_path))
linear_classifier.eval()

# ---------------
# TEST VERİSİ İŞLEME
# ---------------
test_df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/test.csv")  # Sadece filename var
test_filenames = test_df['filename'].values

# Test verisi için embedding çıkar
test_images = []
with torch.no_grad():
    for fname in tqdm(test_filenames):
        img_path = f"/kaggle/input/datathon-ai-qualification-round/test/test/{fname}"
        image = Image.open(img_path).convert("RGB")
        img_tensor = preprocess(image).unsqueeze(0).to(device)
        
        image_features = model.encode_image(img_tensor)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        test_images.append(image_features.cpu())

test_images = torch.cat(test_images, dim=0)

with torch.no_grad():
    test_outputs = linear_classifier(test_images.float().to(device))
    test_pred_idx = test_outputs.argmax(dim=1).cpu().numpy()
    test_pred_cities = [cities[idx] for idx in test_pred_idx]

# Sonuçları submit.csv'ye yaz
submission_df = test_df.copy()
submission_df["city"] = test_pred_cities
submission_df.to_csv("/kaggle/working/submit.csv", index=False)

print("submit.csv dosyası oluşturuldu. En iyi modelle test tahmini tamamlandı.")

/tmp/ipykernel_23/1506039412.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  linear_classifier.load_state_dict(torch.load(model_path))
100%|██████████| 2000/2000 [00:51<

submit.csv dosyası oluşturuldu. En iyi modelle test tahmini tamamlandı.


# OPENAI CLIP LINEAR MLP VB HİPERPARAMETRE DEĞİŞTİREN

In [ ]:
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import clip

############################
# Data Loading and Dataset #
############################

class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, label


###########################
# Classifier Architectures#
###########################

class LinearClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


###########################
# Training/Evaluation Code#
###########################

def extract_embeddings(model, preprocess, image_paths, labels, batch_size=32, device="cuda"):
    dataset = ImageDataset(image_paths, labels, preprocess)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    all_image_features = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        for imgs, labs in tqdm(dataloader):
            imgs = imgs.to(device)
            image_features = model.encode_image(imgs)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            all_image_features.append(image_features.cpu())
            all_labels.append(labs)

    all_image_features = torch.cat(all_image_features, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    return all_image_features, all_labels


def train_and_evaluate(model_name,
                       train_df_path,
                       train_img_dir,
                       prompt_template="A photo of Türkiye in {}.",
                       classifier_type="linear",  # "linear" or "mlp"
                       lr=1e-3,
                       optimizer_type="adam",  # "adam", "adamw", "sgd"
                       epochs=50,
                       hidden_dim=512,
                       weight_decay=0.0,
                       early_stopping=False,
                       patience=10,
                       device="cuda",
                      just_name="zor"):

    # Load CLIP model
    clip_model, preprocess = clip.load(model_name, device=device)
    clip_model.eval()

    # Load train data
    df = pd.read_csv(train_df_path)
    cities = df['city'].unique().tolist()
    city_to_idx = {c: i for i, c in enumerate(cities)}
    labels = df['city'].apply(lambda x: city_to_idx[x]).values
    image_paths = df['filename'].apply(lambda x: f"{train_img_dir}/{x}").values

    # Optionally change prompts if doing zero-shot or embedding text differently
    # (For now we don't do prompt learning here, just a placeholder if needed)

    # Extract embeddings
    all_image_features, all_labels = extract_embeddings(clip_model, preprocess, image_paths, labels, device=device)

    # Split train/val
    num_samples = len(all_image_features)
    train_size = int(0.9 * num_samples)
    val_size = num_samples - train_size

    train_features = all_image_features[:train_size]
    train_labels = all_labels[:train_size]

    val_features = all_image_features[train_size:]
    val_labels = all_labels[train_size:]

    feature_dim = train_features.shape[1]
    num_classes = len(cities)

    # Classifier
    if classifier_type == "linear":
        classifier = LinearClassifier(feature_dim, num_classes).to(device)
    else:
        classifier = MLPClassifier(feature_dim, hidden_dim, num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    if optimizer_type == "adam":
        opt = optim.Adam(classifier.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_type == "adamw":
        opt = optim.AdamW(classifier.parameters(), lr=lr, weight_decay=weight_decay)
    else:  # sgd
        opt = optim.SGD(classifier.parameters(), lr=lr, momentum=0.98, weight_decay=weight_decay)

    best_f1 = 0.0
    no_improve_count = 0
    model_path = f"/kaggle/working/best_model_{just_name}.pt"  # Kaydetmek istediğiniz dosya adı
    
    for epoch in range(epochs):
        classifier.train()
        permutation = torch.randperm(train_size)
        train_features_shuf = train_features[permutation]
        train_labels_shuf = train_labels[permutation]
    
        batch_size=32
        for i in range(0, train_size, batch_size):
            batch_feats = train_features_shuf[i:i+batch_size].to(device).float()
            batch_labs = train_labels_shuf[i:i+batch_size].to(device)
    
            opt.zero_grad()
            outputs = classifier(batch_feats)
            loss = criterion(outputs, batch_labs)
            loss.backward()
            opt.step()
    
        # Validation
        classifier.eval()
        with torch.no_grad():
            val_out = classifier(val_features.float().to(device))
            val_pred = val_out.argmax(dim=1).cpu()
            f1 = f1_score(val_labels, val_pred, average='macro')
    
       
        
        # En iyi modeli kaydet
        if f1 > best_f1:
            best_f1 = f1
            no_improve_count = 0
            torch.save(classifier.state_dict(), model_path)
            print(f"Best model saved with F1: {best_f1:.4f}")
        else:
            no_improve_count += 1
    
        if early_stopping and no_improve_count >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    return best_f1, model_path, feature_dim, hidden_dim, num_classes


###################
# Example Runs     #
###################

# Örnek kullanımlar: Her seferinde sadece bir parametre değiştirip sonucu not edin.
# Bu kodu birden çok kez farklı parametrelerle çalıştırarak deney yapabilirsiniz.



# MLP deneyleri
best_f1, model_path, feature_dim, hidden_dim, num_classes = train_and_evaluate(
    model_name="ViT-L/14@336px",
    train_df_path="/kaggle/input/datathon-ai-qualification-round/train_data.csv",
    train_img_dir="/kaggle/input/datathon-ai-qualification-round/train/train",
    classifier_type="mlp",
    lr=1e-5,
    optimizer_type="sgd",
    epochs=5000,
    hidden_dim=512,
    early_stopping=False,just_name="mlp"
)
print("MLP Classifier F1:", best_f1)


"""

Baseline F1: 0.928632555392408
LR=1e-4 F1: 0.9108111877521411
AdamW F1: 0.9293011149263967
MLP Classifier F1: 0.9399947936750244
AdamW5000 0.9583
"""

In [9]:
device = 'cuda'
model, preprocess = clip.load("ViT-L/14@336px", device=device)  # ya da daha büyük bir model
df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/train_data.csv")
cities = df['city'].unique().tolist()
city_to_idx = {c: i for i, c in enumerate(cities)}
labels = df['city'].apply(lambda x: city_to_idx[x]).values
best_model = MLPClassifier(feature_dim, hidden_dim, num_classes).to(device)
best_model.load_state_dict(torch.load("/kaggle/working/best_model_mlp.pt"))
best_model.eval()


# Test verisini yükle
test_df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/test.csv")  # test.csv sadece filename içeriyor
test_filenames = test_df['filename'].values

test_images = []
with torch.no_grad():
    for fname in tqdm(test_filenames):
        img_path = f"/kaggle/input/datathon-ai-qualification-round/test/test/{fname}"
        image = Image.open(img_path).convert("RGB")
        img_tensor = preprocess(image).unsqueeze(0).to(device)
        
        # CLIP modelinin encode_image fonksiyonunu kullanarak embedding çıkarıyoruz.
        # model değişkeni CLIP modelini temsil ediyor. Eğitimi CLIP embedding'leriyle yaptığımız için testte de aynı şekilde kullanıyoruz.
        image_features = model.encode_image(img_tensor)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        test_images.append(image_features.cpu())

test_images = torch.cat(test_images, dim=0).float().to(device)

with torch.no_grad():
    test_outputs = best_model(test_images)
    test_pred_idx = test_outputs.argmax(dim=1).cpu().numpy()
    test_pred_cities = [cities[idx] for idx in test_pred_idx]

submission_df = test_df.copy()
submission_df["city"] = test_pred_cities
submission_df.to_csv("/kaggle/working/submit.csv", index=False)

print("submit.csv dosyası oluşturuldu. En iyi MLP model ile test tahmini tamamlandı.")


/tmp/ipykernel_23/4082279870.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load("/kaggle/working/best_model_mlp.pt"))
100%|██████████|

submit.csv dosyası oluşturuldu. En iyi MLP model ile test tahmini tamamlandı.


# OPENCLIP

In [2]:
!pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.4 MB/s eta 0:00:00


In [ ]:
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import open_clip

############################
# Data Loading and Dataset #
############################

class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, label


###########################
# Classifier Architectures#
###########################

class LinearClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


###########################
# Training/Evaluation Code#
###########################

def extract_embeddings(model, preprocess, image_paths, labels, batch_size=32, device="cuda"):
    dataset = ImageDataset(image_paths, labels, preprocess)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    all_image_features = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        for imgs, labs in tqdm(dataloader):
            imgs = imgs.to(device)
            image_features = model.encode_image(imgs)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            all_image_features.append(image_features.cpu())
            all_labels.append(labs)

    all_image_features = torch.cat(all_image_features, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    return all_image_features, all_labels


def train_and_evaluate(model_name,
                       pretrained,
                       train_df_path,
                       train_img_dir,
                       prompt_template="A photo of Türkiye in {}.",
                       classifier_type="linear",  # "linear" or "mlp"
                       lr=1e-3,
                       optimizer_type="adam",  # "adam", "adamw", "sgd"
                       epochs=50,
                       hidden_dim=512,
                       weight_decay=0.0,
                       early_stopping=False,
                       patience=10,
                       device="cuda",
                       just_name="zor"):

    # Load open_clip model
    # For example, you can choose model_name = "ViT-L-14" and pretrained = "openai"
    # Refer to open_clip documentation for available model/pretrained combos:
    # https://github.com/mlfoundations/open_clip
    model, preprocess, tokenizer = open_clip.create_model_and_transforms(model_name, pretrained=pretrained, device=device, quick_gelu=True) ## Quick Gelu Düzenlenecek
    model.eval()

    # Load train data
    df = pd.read_csv(train_df_path)
    cities = df['city'].unique().tolist()
    city_to_idx = {c: i for i, c in enumerate(cities)}
    labels = df['city'].apply(lambda x: city_to_idx[x]).values
    image_paths = df['filename'].apply(lambda x: f"{train_img_dir}/{x}").values

    # Extract embeddings
    all_image_features, all_labels = extract_embeddings(model, preprocess, image_paths, labels, device=device)

    # Split train/val
    num_samples = len(all_image_features)
    train_size = int(0.8 * num_samples)
    val_size = num_samples - train_size

    train_features = all_image_features[:train_size]
    train_labels = all_labels[:train_size]

    val_features = all_image_features[train_size:]
    val_labels = all_labels[train_size:]

    feature_dim = train_features.shape[1]
    num_classes = len(cities)

    # Classifier
    if classifier_type == "linear":
        classifier = LinearClassifier(feature_dim, num_classes).to(device)
    else:
        classifier = MLPClassifier(feature_dim, hidden_dim, num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    if optimizer_type == "adam":
        opt = optim.Adam(classifier.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_type == "adamw":
        opt = optim.AdamW(classifier.parameters(), lr=lr, weight_decay=weight_decay)
    else:  # sgd
        opt = optim.SGD(classifier.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)

    best_f1 = 0.0
    no_improve_count = 0
    model_path = f"/kaggle/working/best_model_{just_name}.pt"
    
    for epoch in range(epochs):
        classifier.train()
        permutation = torch.randperm(train_size)
        train_features_shuf = train_features[permutation]
        train_labels_shuf = train_labels[permutation]
    
        batch_size = 32
        for i in range(0, train_size, batch_size):
            batch_feats = train_features_shuf[i:i+batch_size].to(device).float()
            batch_labs = train_labels_shuf[i:i+batch_size].to(device)
    
            opt.zero_grad()
            outputs = classifier(batch_feats)
            loss = criterion(outputs, batch_labs)
            loss.backward()
            opt.step()
    
        # Validation
        classifier.eval()
        with torch.no_grad():
            val_out = classifier(val_features.float().to(device))
            val_pred = val_out.argmax(dim=1).cpu()
            f1 = f1_score(val_labels, val_pred, average='macro')
    
        if f1 > best_f1:
            best_f1 = f1
            no_improve_count = 0
            torch.save(classifier.state_dict(), model_path)
            print(f"Best model saved with F1: {best_f1:.4f}")
        else:
            no_improve_count += 1
    
        if early_stopping and no_improve_count >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    return best_f1, model_path, feature_dim, hidden_dim, num_classes





In [5]:
###################
# Example Run      #
###################

best_f1, model_path, feature_dim, hidden_dim, num_classes = train_and_evaluate(
    model_name="ViT-H-14-378",
    pretrained="dfn5b",  # or another checkpoint available in open_clip
    train_df_path="/kaggle/input/datathon-ai-qualification-round/train_data.csv",
    train_img_dir="/kaggle/input/datathon-ai-qualification-round/train/train",
    classifier_type="mlp",
    lr=1e-3,
    optimizer_type="adam",
    epochs=500,
    hidden_dim=512,
    early_stopping=False,
    just_name="mlp_openclip"
)

print("MLP Classifier F1 with OpenCLIP:", best_f1)

open_clip_pytorch_model.bin:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

100%|██████████| 219/219 [37:26<00:00, 10.26s/it]
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:117: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return F.linear(input, self.weight, self.bias)


Best model saved with F1: 0.9364


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Best model saved with F1: 0.9500


...

Best model saved with F1: 0.9629


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torc

Best model saved with F1: 0.9643


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torc

MLP Classifier F1 with OpenCLIP: 0.9643219030069908


In [10]:
df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/train_data.csv")
cities = df['city'].unique().tolist()
city_to_idx = {c: i for i, c in enumerate(cities)}

# Load the best classifier
best_model = MLPClassifier(feature_dim, hidden_dim, num_classes).to("cuda")
best_model.load_state_dict(torch.load("/kaggle/working/best_model_mlp_openclip.pt"))
best_model.eval()

# Load the CLIP model
clip_model, preprocess, _ = open_clip.create_model_and_transforms(
    "ViT-H-14-378", pretrained="dfn5b", device="cuda"
)
clip_model.eval()

# Test Data Preparation
test_df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/test.csv")  # test.csv has filenames only
test_filenames = test_df['filename'].values

# Extract embeddings for test images
test_images = []
with torch.no_grad():
    for fname in tqdm(test_filenames):
        img_path = f"/kaggle/input/datathon-ai-qualification-round/test/test/{fname}"
        image = Image.open(img_path).convert("RGB")
        img_tensor = preprocess(image).unsqueeze(0).to("cuda")

        # Use CLIP to extract image embeddings (multi-GPU compatible)
        image_features = clip_model.module.encode_image(img_tensor)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        test_images.append(image_features.cpu())

# Combine all test embeddings
test_images = torch.cat(test_images, dim=0).float().to("cuda")

# Generate predictions using the classifier
with torch.no_grad():
    test_outputs = best_model(test_images)
    test_pred_idx = test_outputs.argmax(dim=1).cpu().numpy()
    test_pred_cities = [cities[idx] for idx in test_pred_idx]

# Create and save submission file
submission_df = test_df.copy()
submission_df["city"] = test_pred_cities
submission_df.to_csv("/kaggle/working/submit.csv", index=False)

print("submit.csv created. Predictions completed using the best MLP model.")

/tmp/ipykernel_23/1539157641.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load("/kaggle/working/best_model_mlp_openclip.pt"))
/opt/co

submit.csv created. Predictions completed using the best MLP model.



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


# KFOLD OPENCLIP

In [2]:
!pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00


In [6]:
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import open_clip

############################
# Data Loading and Dataset #
############################

class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, label


###########################
# Classifier Architectures#
###########################

class LinearClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


###########################
# Training/Evaluation Code#
###########################

def extract_embeddings(model, preprocess, image_paths, labels, batch_size=32, device="cuda"):
    dataset = ImageDataset(image_paths, labels, preprocess)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    all_image_features = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        for imgs, labs in tqdm(dataloader):
            imgs = imgs.to(device)
            image_features = model.encode_image(imgs)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            all_image_features.append(image_features.cpu())
            all_labels.append(labs)

    all_image_features = torch.cat(all_image_features, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    return all_image_features, all_labels




def train_and_evaluate_with_kfold(
        model_name,
        pretrained,
        train_df_path,
        train_img_dir,
        prompt_template="A photo of Türkiye in {}.",
        classifier_type="linear",  # "linear" or "mlp"
        lr=1e-3,
        optimizer_type="adam",  # "adam", "adamw", "sgd"
        epochs=50,
        hidden_dim=512,
        weight_decay=0.0,
        early_stopping=False,
        patience=10,
        device="cuda",
        k_folds=5):

    # Load open_clip model
    model, preprocess, tokenizer = open_clip.create_model_and_transforms(model_name, pretrained=pretrained, device=device, quick_gelu=True)
    model.eval()

    # Load train data
    df = pd.read_csv(train_df_path)
    cities = df['city'].unique().tolist()
    city_to_idx = {c: i for i, c in enumerate(cities)}
    labels = df['city'].apply(lambda x: city_to_idx[x]).values
    image_paths = df['filename'].apply(lambda x: f"{train_img_dir}/{x}").values

    # Extract embeddings
    all_image_features, all_labels = extract_embeddings(model, preprocess, image_paths, labels, device=device)

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    all_f1_scores = []
    model_paths = []
    feature_dim = all_image_features.shape[1]
    num_classes = len(cities)
    fold_idx = 0

    for train_idx, val_idx in kf.split(all_image_features):
        print(f"Fold {fold_idx + 1}/{k_folds}")
        fold_idx += 1

        train_features = all_image_features[train_idx]
        train_labels = all_labels[train_idx]
        val_features = all_image_features[val_idx]
        val_labels = all_labels[val_idx]

        # Classifier
        if classifier_type == "linear":
            classifier = LinearClassifier(feature_dim, num_classes).to(device)
        else:
            classifier = MLPClassifier(feature_dim, hidden_dim, num_classes).to(device)

        criterion = nn.CrossEntropyLoss()
        if optimizer_type == "adam":
            opt = optim.Adam(classifier.parameters(), lr=lr, weight_decay=weight_decay)
        elif optimizer_type == "adamw":
            opt = optim.AdamW(classifier.parameters(), lr=lr, weight_decay=weight_decay)
        else:  # sgd
            opt = optim.SGD(classifier.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)

        best_f1 = 0.0
        no_improve_count = 0
        model_path = f"/kaggle/working/best_model_fold_{fold_idx}.pt"

        for epoch in range(epochs):
            classifier.train()
            permutation = torch.randperm(len(train_features))
            train_features_shuf = train_features[permutation]
            train_labels_shuf = train_labels[permutation]

            batch_size = 32
            for i in range(0, len(train_features_shuf), batch_size):
                batch_feats = train_features_shuf[i:i + batch_size].to(device).float()
                batch_labs = train_labels_shuf[i:i + batch_size].to(device)

                opt.zero_grad()
                outputs = classifier(batch_feats)
                loss = criterion(outputs, batch_labs)
                loss.backward()
                opt.step()

            # Validation
            classifier.eval()
            with torch.no_grad():
                val_out = classifier(val_features.float().to(device))
                val_pred = val_out.argmax(dim=1).cpu()
                f1 = f1_score(val_labels, val_pred, average='macro')

            if f1 > best_f1:
                best_f1 = f1
                no_improve_count = 0
                torch.save(classifier.state_dict(), model_path)
                print(f"Best model for fold {fold_idx} saved with F1: {best_f1:.4f}")
            else:
                no_improve_count += 1

            if early_stopping and no_improve_count >= patience:
                print(f"Early stopping at epoch {epoch + 1} for fold {fold_idx}")
                break

        all_f1_scores.append(best_f1)
        model_paths.append(model_path)

    mean_f1 = sum(all_f1_scores) / len(all_f1_scores)
    print(f"Mean F1 Score across {k_folds} folds: {mean_f1:.4f}")

    return all_f1_scores, mean_f1, model_paths, feature_dim, hidden_dim, num_classes


In [7]:
###################
# Example Run      #
###################

all_f1_scores, mean_f1, model_path, feature_dim, hidden_dim, num_classes = train_and_evaluate_with_kfold(
    model_name="ViT-H-14",
    pretrained="dfn5b",  # or another checkpoint available in open_clip
    train_df_path="/kaggle/input/datathon-ai-qualification-round/train_data.csv",
    train_img_dir="/kaggle/input/datathon-ai-qualification-round/train/train",
    classifier_type="mlp",
    lr=1e-3,
    optimizer_type="adam",
    epochs=500,
    hidden_dim=512,
    early_stopping=False,
)

print("MLP Classifier F1 with OpenCLIP:", mean_f1)

open_clip_pytorch_model.bin:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

100%|██████████| 219/219 [13:54<00:00,  3.81s/it]


Fold 1/5
Best model for fold 1 saved with F1: 0.9091
Best model for fold 1 saved with F1: 0.9198
Best model for fold 1 saved with F1: 0.9262
Best model for fold 1 saved with F1: 0.9375
Best model for fold 1 saved with F1: 0.9399
Best model for fold 1 saved with F1: 0.9406
Best model for fold 1 saved with F1: 0.9434
Best model for fold 1 saved with F1: 0.9449
Best model for fold 1 saved with F1: 0.9471
Best model for fold 1 saved with F1: 0.9471
Best model for fold 1 saved with F1: 0.9478
Best model for fold 1 saved with F1: 0.9486
Best model for fold 1 saved with F1: 0.9515
Best model for fold 1 saved with F1: 0.9515
Best model for fold 1 saved with F1: 0.9528
Best model for fold 1 saved with F1: 0.9528
Best model for fold 1 saved with F1: 0.9529
Best model for fold 1 saved with F1: 0.9536
Best model for fold 1 saved with F1: 0.9565
Fold 2/5
Best model for fold 2 saved with F1: 0.9042
Best model for fold 2 saved with F1: 0.9224
Best model for fold 2 saved with F1: 0.9238
Best model for

# PROMPT + OPENCLIP + KFOLD

In [1]:
!pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00


In [3]:
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import open_clip
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, ReduceLROnPlateau, OneCycleLR
import math

############################
# Data Loading and Dataset #
############################

class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, label


###########################
# Classifier Architectures#
###########################

class LinearClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


def train_and_evaluate_with_kfold(
        model_name,
        pretrained,
        train_df_path,
        train_img_dir,
        # prompt_template kullanmayacağız, onun yerine sabit prompt listesi
        classifier_type="linear",  # "linear" or "mlp"
        lr=1e-3,
        optimizer_type="adam",  # "adam", "adamw", "sgd"
        momentum=0.9,  # SGD için momentum
        scheduler_type=None,  # "steplr", "cosine", "plateau", "onecycle"
        step_size=10,
        gamma=0.1,
        epochs=50,
        hidden_dim=256,
        weight_decay=0.0,
        early_stopping=False,
        patience=10,
        device="cuda",
        k_folds=10):

    # **YENİ EKLENDİ**: Birden fazla prompt
    prompts = [
        "A daytime photo of Türkiye.",
        "A detailed view of a city in Türkiye.",
        "An urban photograph in Türkiye."
    ]

    # Load open_clip model
    model, preprocess, tokenizer = open_clip.create_model_and_transforms(model_name, pretrained=pretrained, device=device)
    model.eval()

    # Load train data
    df = pd.read_csv(train_df_path)
    cities = df['city'].unique().tolist()
    city_to_idx = {c: i for i, c in enumerate(cities)}
    labels = df['city'].apply(lambda x: city_to_idx[x]).values
    image_paths = df['filename'].apply(lambda x: f"{train_img_dir}/{x}").values

    # **YENİ EKLENDİ**: Ensemble prompt text embedding
    with torch.no_grad():
        text_tokens = open_clip.tokenize(prompts).to(device)
        text_features = model.encode_text(text_tokens)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        # Ortalamasını al
        ensemble_text_feature = text_features.mean(dim=0)  # (D,)

    # Extract embeddings (sadece image)
    def extract_embeddings(model, preprocess, image_paths, labels, device="cuda"):
        dataset = ImageDataset(image_paths, labels, preprocess)
        dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

        all_image_features = []
        all_labels = []

        model.eval()
        with torch.no_grad():
            for imgs, labs in tqdm(dataloader):
                imgs = imgs.to(device)
                image_features = model.encode_image(imgs)
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)
                all_image_features.append(image_features.cpu())
                all_labels.append(labs)

        all_image_features = torch.cat(all_image_features, dim=0)
        all_labels = torch.cat(all_labels, dim=0)
        return all_image_features, all_labels

    all_image_features, all_labels = extract_embeddings(model, preprocess, image_paths, labels, device=device)

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    all_f1_scores = []
    model_paths = []
    feature_dim = all_image_features.shape[1]
    num_classes = len(cities)
    fold_idx = 0

    for train_idx, val_idx in kf.split(all_image_features):
        print(f"Fold {fold_idx + 1}/{k_folds}")
        fold_idx += 1

        train_features = all_image_features[train_idx].to(device)
        train_labels = all_labels[train_idx]
        val_features = all_image_features[val_idx].to(device)
        val_labels = all_labels[val_idx]

        # Her imaj için ensemble_text_feature'ı ekle
        text_feature_expanded_train = ensemble_text_feature.unsqueeze(0).expand(train_features.shape[0], -1)
        text_feature_expanded_val = ensemble_text_feature.unsqueeze(0).expand(val_features.shape[0], -1)

        combined_train_features = torch.cat((train_features, text_feature_expanded_train), dim=1)
        combined_val_features = torch.cat((val_features, text_feature_expanded_val), dim=1)

        combined_feature_dim = combined_train_features.shape[1]

        # Classifier
        if classifier_type == "linear":
            classifier = LinearClassifier(combined_feature_dim, num_classes).to(device)
        else:
            classifier = MLPClassifier(combined_feature_dim, hidden_dim, num_classes).to(device)

        criterion = nn.CrossEntropyLoss()
        if optimizer_type == "adam":
            opt = optim.Adam(classifier.parameters(), lr=lr, weight_decay=weight_decay)
        elif optimizer_type == "adamw":
            opt = optim.AdamW(classifier.parameters(), lr=lr, weight_decay=weight_decay)
        else:  # sgd
            opt = optim.SGD(classifier.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

        # **YENİ EKLENDİ** OneCycleLR için adımlar
        # OneCycleLR için epoch başına adım sayısını hesapla
        batch_size = 32
        steps_per_epoch = math.ceil(len(combined_train_features) / batch_size)

        if scheduler_type == "steplr":
            scheduler = StepLR(opt, step_size=step_size, gamma=gamma)
        elif scheduler_type == "cosine":
            scheduler = CosineAnnealingLR(opt, T_max=epochs)
        elif scheduler_type == "plateau":
            scheduler = ReduceLROnPlateau(opt, mode="max", factor=gamma, patience=patience)
        elif scheduler_type == "onecycle":
            # OneCycleLR ile max_lr'ı belirleyin. Burada lr'nin 10 katını denedik örnek olarak.
            scheduler = OneCycleLR(opt, max_lr=lr*10, steps_per_epoch=steps_per_epoch, epochs=epochs)
        else:
            scheduler = None

        best_f1 = 0.0
        no_improve_count = 0
        model_path = f"/kaggle/working/best_model_fold_{fold_idx}.pt"

        for epoch in range(epochs):
            classifier.train()
            permutation = torch.randperm(len(combined_train_features))
            train_features_shuf = combined_train_features[permutation]
            train_labels_shuf = train_labels[permutation]

            for i in range(0, len(train_features_shuf), batch_size):
                batch_feats = train_features_shuf[i:i + batch_size].float()
                batch_labs = train_labels_shuf[i:i + batch_size].to(device)

                opt.zero_grad()
                outputs = classifier(batch_feats)
                loss = criterion(outputs, batch_labs)
                loss.backward()
                opt.step()

                # **YENİ EKLENDİ**: OneCycleLR her iterasyonda step
                if scheduler_type == "onecycle":
                    scheduler.step()

            # Validation
            classifier.eval()
            with torch.no_grad():
                val_out = classifier(combined_val_features.float())
                val_pred = val_out.argmax(dim=1).cpu()
                f1 = f1_score(val_labels, val_pred, average='macro')

            # Diğer scheduler'lar epoch sonunda step alabilir
            if scheduler is not None and scheduler_type != "onecycle":
                if scheduler_type == "plateau":
                    scheduler.step(f1) 
                else:
                    scheduler.step()
            if f1 % 50 == 0:
                print(f"f1: {f1} in epoch: {epoch}")
            if f1 > best_f1:
                best_f1 = f1
                no_improve_count = 0
                torch.save(classifier.state_dict(), model_path)
                print(f"Best model for fold {fold_idx} saved with F1: {best_f1:.4f}")
            else:
                no_improve_count += 1

            if early_stopping and no_improve_count >= patience:
                print(f"Early stopping at epoch {epoch + 1} for fold {fold_idx}")
                break

        all_f1_scores.append(best_f1)
        model_paths.append(model_path)

    mean_f1 = sum(all_f1_scores) / len(all_f1_scores)
    print(f"Mean F1 Score across {k_folds} folds: {mean_f1:.4f}")

    return all_f1_scores, mean_f1, model_paths, combined_feature_dim, hidden_dim, num_classes


In [ ]:
###################
# Example Run      #
###################

MODEL_TYPE = "ViT-H-14-378-quickgelu"
MODEL_PRETRAIN_DATA = "dfn5b"


all_f1_scores, mean_f1, model_path, combined_feature_dim, hidden_dim, num_classes = train_and_evaluate_with_kfold(
    model_name=MODEL_TYPE,
    pretrained=MODEL_PRETRAIN_DATA,  # or another checkpoint available in open_clip
    train_df_path="/kaggle/input/datathon-ai-qualification-round/train_data.csv",
    train_img_dir="/kaggle/input/datathon-ai-qualification-round/train/train",
    classifier_type="mlp",
    lr=1e-3,
    optimizer_type="sgd",
    epochs=100,
    hidden_dim=1024,
    early_stopping=False,
    scheduler_type='plateau',
    k_folds=5
)

print("MLP Classifier F1 with OpenCLIP:", mean_f1)

In [ ]:
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
import open_clip


MODEL_PATH = "/kaggle/working/best_model_fold_2.pt"

# Birden fazla prompt kullanımı
prompts = [
    "A daytime photo of Türkiye.",
    "A detailed view of a city in Türkiye.",
    "An urban photograph in Türkiye."
]

df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/train_data.csv")
cities = df['city'].unique().tolist()
city_to_idx = {c: i for i, c in enumerate(cities)}

# Load the best classifier
best_model = MLPClassifier(combined_feature_dim, hidden_dim, num_classes).to("cuda")
best_model.load_state_dict(torch.load(MODEL_PATH))
best_model.eval()

# Load the CLIP model - aynı model ve checkpoint'i kullanın
clip_model, preprocess, _ = open_clip.create_model_and_transforms(
    MODEL_TYPE, pretrained=MODEL_PRETRAIN_DATA, device="cuda")
clip_model.eval()

# Birden fazla prompt'tan ensemble text embedding elde et
with torch.no_grad():
    text_tokens = open_clip.tokenize(prompts).to("cuda")
    text_features = clip_model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    # Promptların ortalamasını al
    ensemble_text_feature = text_features.mean(dim=0)  # (D,)

test_df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/test.csv")
test_filenames = test_df['filename'].values

test_images = []
with torch.no_grad():
    for fname in tqdm(test_filenames):
        img_path = f"/kaggle/input/datathon-ai-qualification-round/test/test/{fname}"
        image = Image.open(img_path).convert("RGB")
        img_tensor = preprocess(image).unsqueeze(0).to("cuda")

        # Eğer clip_model DataParallel ile sarılı değilse .module'a gerek yoktur:
        image_features = clip_model.encode_image(img_tensor)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        test_images.append(image_features)

# Combine all test embeddings
test_images = torch.cat(test_images, dim=0).float().to("cuda")

# Her test imajına ensemble_text_feature'ı ekle
text_feature_expanded = ensemble_text_feature.unsqueeze(0).expand(test_images.shape[0], -1)
test_combined = torch.cat((test_images, text_feature_expanded), dim=1)

# Generate predictions using the classifier
with torch.no_grad():
    test_outputs = best_model(test_combined)
    test_pred_idx = test_outputs.argmax(dim=1).cpu().numpy()
    test_pred_cities = [cities[idx] for idx in test_pred_idx]

# Create and save submission file
submission_df = test_df.copy()
submission_df["city"] = test_pred_cities
submission_df.to_csv("/kaggle/working/submit.csv", index=False)

print("submit.csv created. Predictions completed using the best MLP model with ensemble text embedding.")


In [ ]:
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"
quick_gelu = False
truncate=True # BAZI MODELLERDE TOKEN SIZE SINIRI VAR
model_name = "ViT-H-14-378-quickgelu"
pretrained = "dfn5b"
model, preprocess, tokenizer = open_clip.create_model_and_transforms(model_name, pretrained=pretrained, device=device)
model.eval()

# Veri Yükleme
df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/train_data.csv")
cities = df['city'].unique().tolist()

# Her şehir için birden fazla prompt tanımlayalım.
# Örneğin, şehir ismini hafif varyasyonlarla ifade edelim. 
# Bu, modelin mükemmel bir şekilde etiketleri tahmin etmesini zorlaştırabilir.
prompts_per_city = [
    "A daytime photo of {}, Türkiye.",
    "A detailed urban view in {} city, Türkiye.",
    "An architectural glance of {}, located in Türkiye."
]

# Şehir bazında ensemble text embedding oluşturma
city_text_features = []
with torch.no_grad():
    for city in cities:
        city_prompts = [p.format(city) for p in prompts_per_city]
        text_tokens = open_clip.tokenize(city_prompts).to(device)
        text_feats = model.encode_text(text_tokens)
        text_feats = text_feats / text_feats.norm(dim=-1, keepdim=True)
        # Ortalama alarak ensemble elde ediyoruz
        city_feature = text_feats.mean(dim=0)  # (D,) boyutunda
        city_text_features.append(city_feature)
        
city_text_features = torch.stack(city_text_features, dim=0)  # (num_cities, D)

# Zero-Shot Sınıflandırma Testi
correct = 0
for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_path = row['filename']
    label = row['city']
    image_path = f"/kaggle/input/datathon-ai-qualification-round/train/train/{img_path}"
    image = Image.open(image_path).convert("RGB")
    image_input = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_input)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)

    # Benzerlik hesabı
    similarity = (image_features @ city_text_features.T).squeeze(0)
    pred_idx = similarity.argmax().item()
    pred_city = cities[pred_idx]

    if pred_city == label:
        correct += 1

accuracy = correct / len(df)
print("Zero-Shot Accuracy:", accuracy)


# PROMPT + OPENCLIP + KFOLD + AUGMENT

In [1]:
!pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00


In [ ]:
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import open_clip
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math
from torchvision import transforms

############################
# Data Loading and Dataset #
############################

class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        return image, label


###########################
# Classifier Architectures#
###########################

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


def train_and_evaluate_with_kfold(
        model_name,
        pretrained,
        train_df_path,
        train_img_dir,
        prompts,
        lr=1e-5,
        optimizer_type="adamw",
        scheduler_type="plateau",
        epochs=20,
        hidden_dim=1024,
        weight_decay=0.01,
        early_stopping=True,
        patience=5,
        device="cuda",
        k_folds=5):

    # Load train data
    df = pd.read_csv(train_df_path)
    cities = df['city'].unique().tolist()
    city_to_idx = {c: i for i, c in enumerate(cities)}
    labels = df['city'].apply(lambda x: city_to_idx[x]).values
    image_paths = df['filename'].apply(lambda x: f"{train_img_dir}/{x}").values

    # CLIP model load
    model, preprocess_base, tokenizer = open_clip.create_model_and_transforms(model_name, pretrained=pretrained, device=device)
    model.eval()

    # Prompt text features (isteğe bağlı)
    with torch.no_grad():
        text_tokens = open_clip.tokenize(prompts).to(device)
        text_features = model.encode_text(text_tokens)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        ensemble_text_feature = text_features.mean(dim=0)  # (D,)

    # Training augmentations (modelin beklendiği çözünürlükle uyumlu)
    # Örneğin ViT-H-14-378 için 378x378
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.48145466, 0.4578275, 0.40821073],
            std=[0.26862954, 0.26130258, 0.27577711]
        )
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.48145466, 0.4578275, 0.40821073],
            std=[0.26862954, 0.26130258, 0.27577711]
        )
    ])

    # Öncelikle feature boyutunu bulmak için modelden bir batch geçirelim
    # Bu sayede combined_feature_dim = image_feature_dim + text_feature_dim hesaplanır.
    # text_features boyutu: D
    text_dim = ensemble_text_feature.shape[0]

    # Geçici olarak bir görüntü alalım
    temp_img = Image.open(image_paths[0]).convert("RGB")
    temp_img = val_transform(temp_img).unsqueeze(0).to(device)
    with torch.no_grad():
        img_feat = model.encode_image(temp_img)
        img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)
    image_dim = img_feat.shape[1]

    combined_feature_dim = image_dim + text_dim
    num_classes = len(cities)

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    all_f1_scores = []
    model_paths = []
    fold_idx = 0

    for train_idx, val_idx in kf.split(image_paths):
        fold_idx += 1
        print(f"Fold {fold_idx}/{k_folds}")

        train_dataset = ImageDataset(image_paths[train_idx], labels[train_idx], transform=train_transform)
        val_dataset = ImageDataset(image_paths[val_idx], labels[val_idx], transform=val_transform)

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

        # Classifier
        classifier = MLPClassifier(combined_feature_dim, hidden_dim, num_classes).to(device)

        # Unfreeze bazı katmanlar isterseniz burada yapın
        # Örn: model parameters freeze/unfreeze
        for param in model.parameters():
            param.requires_grad = False
        # Burada örnek olarak hiçbir katman açmıyoruz.
        # Eğer fine-tune edecekseniz:
        #num_blocks = len(model.visual.transformer.resblocks)
        #unfreeze_blocks = 2
        #for i in range(num_blocks - unfreeze_blocks, num_blocks):
        #    for param in model.visual.transformer.resblocks[i].parameters():
        #        param.requires_grad = True
        #for param in model.visual.ln_post.parameters():
        #    param.requires_grad = True

        params_to_optimize = list(classifier.parameters())  # Fine-tune etmezsek sadece classifier optimize
        if optimizer_type == "adamw":
            opt = optim.AdamW(params_to_optimize, lr=lr, weight_decay=weight_decay)
        else:
            opt = optim.Adam(params_to_optimize, lr=lr, weight_decay=weight_decay)

        if scheduler_type == "plateau":
            scheduler = ReduceLROnPlateau(opt, mode="max", factor=0.5, patience=3)
        else:
            scheduler = None

        best_f1 = 0.0
        no_improve_count = 0
        model_path = f"/kaggle/working/best_model_fold_{fold_idx}.pt"

        for epoch in range(epochs):
            classifier.train()
            model.eval()  # CLIP'i freeze ettiğimiz için eval modda tutuyoruz
            for imgs, labs in tqdm(train_loader, desc=f"Fold {fold_idx}, Epoch {epoch+1}/{epochs} [Train]"):
                imgs = imgs.to(device)
                labs = labs.to(device)

                opt.zero_grad()
                with torch.no_grad():
                    image_features = model.encode_image(imgs)
                    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
                # combine features
                text_feature_expanded = ensemble_text_feature.unsqueeze(0).expand(image_features.shape[0], -1)
                combined_features = torch.cat((image_features, text_feature_expanded), dim=1)
                outputs = classifier(combined_features)
                loss = nn.CrossEntropyLoss()(outputs, labs)
                loss.backward()
                opt.step()

            # Validation
            classifier.eval()
            val_preds = []
            val_trues = []
            with torch.no_grad():
                for imgs, labs in tqdm(val_loader, desc=f"Fold {fold_idx}, Epoch {epoch+1}/{epochs} [Val]"):
                    imgs = imgs.to(device)
                    labs = labs.to(device)
                    image_features = model.encode_image(imgs)
                    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
                    text_feature_expanded = ensemble_text_feature.unsqueeze(0).expand(image_features.shape[0], -1)
                    combined_features = torch.cat((image_features, text_feature_expanded), dim=1)
                    val_out = classifier(combined_features)
                    val_pred = val_out.argmax(dim=1).cpu()
                    val_preds.append(val_pred)
                    val_trues.append(labs.cpu())

            val_preds = torch.cat(val_preds)
            val_trues = torch.cat(val_trues)
            f1 = f1_score(val_trues, val_preds, average='macro')
            print(f"Fold {fold_idx}, Epoch {epoch+1}, F1: {f1:.4f}")

            if scheduler is not None:
                scheduler.step(f1)

            if f1 > best_f1:
                best_f1 = f1
                no_improve_count = 0
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'classifier_state_dict': classifier.state_dict()
                }, model_path)
                print(f"Best model updated at Fold {fold_idx}, Epoch {epoch+1}, F1: {best_f1:.4f}")
            else:
                no_improve_count += 1

            if early_stopping and no_improve_count >= patience:
                print(f"Early stopping at epoch {epoch+1} for fold {fold_idx}")
                break

        all_f1_scores.append(best_f1)
        model_paths.append(model_path)

    mean_f1 = sum(all_f1_scores) / len(all_f1_scores)
    print(f"Mean F1 Score across {k_folds} folds: {mean_f1:.4f}")

    return all_f1_scores, mean_f1, model_paths, combined_feature_dim, hidden_dim, num_classes


###################
# Example Run      #
###################

MODEL_TYPE = "ViT-g-14"
MODEL_PRETRAIN_DATA = "laion2b_s12b_b42k"

prompts = [
    "A daytime urban photograph in Türkiye."
]

all_f1_scores, mean_f1, model_paths, combined_feature_dim, hidden_dim, num_classes = train_and_evaluate_with_kfold(
    model_name=MODEL_TYPE,
    pretrained=MODEL_PRETRAIN_DATA,
    train_df_path="/kaggle/input/datathon-ai-qualification-round/train_data.csv",
    train_img_dir="/kaggle/input/datathon-ai-qualification-round/train/train",
    prompts=prompts,
    lr=1e-5,
    optimizer_type="adamw",
    epochs=10,
    hidden_dim=128,
    early_stopping=True,
    patience=5,
    scheduler_type='plateau',
    k_folds=5
)

print("Training completed.")
print("Mean F1:", mean_f1)
print("Model paths:", model_paths)


In [ ]:
aaaaaaaaaaaaaaaaa

In [ ]:
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
import open_clip
from torchvision import transforms

# Aynı classifier mimarisi
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

DEVICE = "cuda"
TEST_MODEL_PATH = "/kaggle/working/best_model_fold_2.pt"  # Seçtiğiniz en iyi modeli belirtin
MODEL_TYPE = "ViT-H-14-378-quickgelu"
MODEL_PRETRAIN_DATA = "dfn5b"

# Eğitimden elde ettiğiniz bu değerleri kullanın:
combined_feature_dim = combined_feature_dim 
hidden_dim = 1024
num_classes = 3  # şehir sayısı
prompts = [
    "A daytime photo of Türkiye.",
    "A detailed view of a city in Türkiye.",
    "An urban photograph in Türkiye."
]

train_df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/train_data.csv")
cities = train_df['city'].unique().tolist()

test_df = pd.read_csv("/kaggle/input/datathon-ai-qualification-round/test.csv")
test_filenames = test_df['filename'].values

# Modeli yükle
model, preprocess_base, tokenizer = open_clip.create_model_and_transforms(
    MODEL_TYPE, pretrained=MODEL_PRETRAIN_DATA, device=DEVICE
)
model.eval()

classifier = MLPClassifier(combined_feature_dim, hidden_dim, num_classes).to(DEVICE)

# Kaydedilen model ağırlıklarını yükle
checkpoint = torch.load(TEST_MODEL_PATH, map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
classifier.load_state_dict(checkpoint['classifier_state_dict'])
classifier.eval()

# Text embedding
with torch.no_grad():
    text_tokens = open_clip.tokenize(prompts).to(DEVICE)
    text_features = model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    ensemble_text_feature = text_features.mean(dim=0)  # (D,)

# Test transform
test_transform = transforms.Compose([
    transforms.Resize((378, 378)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.48145466, 0.4578275, 0.40821073],
        std=[0.26862954, 0.26130258, 0.27577711]
    )
])

test_images = []
with torch.no_grad():
    for fname in tqdm(test_filenames):
        img_path = f"/kaggle/input/datathon-ai-qualification-round/test/test/{fname}"
        image = Image.open(img_path).convert("RGB")
        img_tensor = test_transform(image).unsqueeze(0).to(DEVICE)

        image_features = model.encode_image(img_tensor)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        test_images.append(image_features)

test_images = torch.cat(test_images, dim=0).float().to(DEVICE)

# Metin featurelarını ekle
text_feature_expanded = ensemble_text_feature.unsqueeze(0).expand(test_images.shape[0], -1)
test_combined = torch.cat((test_images, text_feature_expanded), dim=1)

# Tahmin
with torch.no_grad():
    test_outputs = classifier(test_combined)
    test_pred_idx = test_outputs.argmax(dim=1).cpu().numpy()
    test_pred_cities = [cities[idx] for idx in test_pred_idx]

# Submit
submission_df = test_df.copy()
submission_df["city"] = test_pred_cities
submission_df.to_csv("/kaggle/working/submit.csv", index=False)

print("submit.csv created. Predictions completed.")


# TEMIZ CLIP

In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-crzznkz6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-crzznkz6
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=4bcc28c3e97b1fc5dc2bdd81f35c4e73de4f3ab667e2a833e34367b55fb68066
  Stored in directory: /tmp/pip-ephem-wheel-cache-jpe00i8x/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random


import clip

#------------------------------------------------------------
# Config
#------------------------------------------------------------
IMG_DIR = "/kaggle/input/datathon-ai-qualification-round/train/train"  # resimlerin bulunduğu klasör
CSV_PATH = "/kaggle/input/datathon-ai-qualification-round/train_data.csv" # filename, city içeriyor
NUM_FOLDS = 3
SEED = 42
BATCH_SIZE = 16
EPOCHS = 10
LR = 1e-6  # Learning rate'i biraz düşürün, artık daha fazla katman eğitiliyor.
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
CLIP_MODEL_NAME = "ViT-B/16"
NUM_WORKERS = 4
NUM_CLASSES = 3
unfreeze_layers = 0  # Son 2 transformer bloğunu tune edelim.

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(SEED)

#------------------------------------------------------------
# Dataset
#------------------------------------------------------------
class CityDataset(Dataset):
    def __init__(self, df, img_dir, transforms=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = self.read_image(img_path)
        label = row['city_label']
        
        if self.transforms:
            image = self.transforms(image=image)['image']
        return image, label

    @staticmethod
    def read_image(path):
        import cv2
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

#------------------------------------------------------------
# CLIP Normalizasyon
#------------------------------------------------------------
CLIP_MEAN = (0.48145466, 0.4578275, 0.40821073)
CLIP_STD = (0.26862954, 0.26130258, 0.27577711)

def get_transforms(train=True):
    if train:
        return A.Compose([
            A.RandomResizedCrop(height=224, width=224, scale=(0.8,1.0)),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=CLIP_MEAN, std=CLIP_STD),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(224,224),
            A.Normalize(mean=CLIP_MEAN, std=CLIP_STD),
            ToTensorV2(),
        ])

#------------------------------------------------------------
# Model
#------------------------------------------------------------
class CLIPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super(CLIPClassifier, self).__init__()
        self.clip_model = clip_model.float()
        # CLIP modeli genellikle özellik boyutu ViT-B/32 için 512, ViT-L/14 için 768'dir.
        # Aşağıda projeyi kontrol ediyoruz.
        if hasattr(self.clip_model.visual, 'proj') and self.clip_model.visual.proj is not None:
            embed_dim = self.clip_model.visual.proj.shape[1]
        else:
            # Fallback değeri
            embed_dim = 512 
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        image_features = self.clip_model.encode_image(x)
        logits = self.classifier(image_features)
        return logits

def get_model(num_classes, device, unfreeze_layers=2):
    clip_model, preprocess = clip.load(CLIP_MODEL_NAME, device=device)

    # Tüm katmanları dondur
    for param in clip_model.parameters():
        param.requires_grad = False

    # Görsel encoder: clip_model.visual
    # Bu genellikle bir VisionTransformer. Son N bloğu açalım.
    # clip_model.visual.transformer.resblocks: bir ModuleList 12 blok içerir (ViT-B/32 için).
    total_blocks = len(clip_model.visual.transformer.resblocks)
    # Son 'unfreeze_layers' bloğu aç
    for i in range(total_blocks - unfreeze_layers, total_blocks):
        for param in clip_model.visual.transformer.resblocks[i].parameters():
            param.requires_grad = True

    # Son olarak linear classifier serbest: 
    # Zaten classifier bizim eklediğimiz katman, varsayılan olarak requires_grad=True olacak.
    model = CLIPClassifier(clip_model, num_classes).to(device)

    # İsterseniz LayerNorm, positional embedding gibi bazı kısımları da açabilirsiniz:
    # Örnek: 
    # for param in clip_model.visual.ln_post.parameters():
    #     param.requires_grad = True
    #
    # Bu şekilde ince ayarlar yapabilirsiniz.

    return model

#------------------------------------------------------------
# Eğitim/Val Fonksiyonları
#------------------------------------------------------------
def train_one_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(dataloader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += loss.item() * labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

def validate_one_epoch(model, dataloader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            preds = outputs.argmax(dim=1)

            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()*labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total

    from sklearn.metrics import f1_score
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return avg_loss, accuracy, f1

#------------------------------------------------------------
# K-Fold Training
#------------------------------------------------------------
df = pd.read_csv(CSV_PATH)
cities = df['city'].unique()
city_to_idx = {c: i for i,c in enumerate(cities)}
df['city_label'] = df['city'].map(city_to_idx)

X = df['filename'].values
y = df['city_label'].values

skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"========== Fold {fold+1}/{NUM_FOLDS} ==========")
    train_df = df.iloc[train_idx].reset_index(drop=True)
    val_df = df.iloc[val_idx].reset_index(drop=True)

    train_dataset = CityDataset(train_df, IMG_DIR, transforms=get_transforms(train=True))
    val_dataset = CityDataset(val_df, IMG_DIR, transforms=get_transforms(train=False))

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    model = get_model(NUM_CLASSES, DEVICE, unfreeze_layers=unfreeze_layers)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LR, steps_per_epoch=len(train_loader), epochs=EPOCHS)

    best_f1 = 0.0
    best_model_path = f"best_clip_model_fold{fold}.pth"

    for epoch in range(EPOCHS):
        print(f"Epoch {epoch+1}/{EPOCHS}")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, scheduler, DEVICE)
        val_loss, val_acc, val_f1 = validate_one_epoch(model, val_loader, DEVICE)

        print(f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} F1: {val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), best_model_path)

    fold_results.append(best_f1)

print("K-Fold Sonuçları:")
for i, r in enumerate(fold_results):
    print(f"Fold {i+1}: F1 = {r:.4f}")
print(f"Ortalama F1 = {np.mean(fold_results):.4f}")


In [ ]:
aaaaaaaaa

In [4]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import clip
import torch.nn as nn

# Aynı model adıyla yükleyin
CLIP_MODEL_NAME = "ViT-L/14"  # Eğitimde ne kullandıysanız onu yazın
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
NUM_WORKERS = 4

TEST_IMG_DIR = "/kaggle/input/datathon-ai-qualification-round/test/test"   # Test resimlerinin olduğu klasör
TEST_CSV_PATH = "/kaggle/input/datathon-ai-qualification-round/test.csv"
MODEL_PATH = "/kaggle/working/best_clip_model_fold1.pth"  # Eğitilen model checkpoint'i
TRAIN_CSV_PATH = "/kaggle/input/datathon-ai-qualification-round/train_data.csv" # cities bilgisini buradan alacağız

# train.csv'den city bilgilerini al
train_df = pd.read_csv(TRAIN_CSV_PATH)
cities = train_df['city'].unique()
city_to_idx = {c: i for i,c in enumerate(cities)}
num_classes = len(cities)

test_df = pd.read_csv(TEST_CSV_PATH)

CLIP_MEAN = (0.48145466, 0.4578275, 0.40821073)
CLIP_STD = (0.26862954, 0.26130258, 0.27577711)

def get_test_transforms():
    return A.Compose([
        A.Resize(224,224),
        A.Normalize(mean=CLIP_MEAN, std=CLIP_STD),
        ToTensorV2(),
    ])

class TestDataset(Dataset):
    def __init__(self, df, img_dir, transforms=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        filename = row['filename']
        img_path = os.path.join(self.img_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transforms:
            img = self.transforms(image=img)['image']
        return img, filename

class CLIPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super(CLIPClassifier, self).__init__()
        self.clip_model = clip_model.float()
        # ViT-B/32 = 512 dim, ViT-L/14 = 768 dim gibi farklı olabilir
        if hasattr(self.clip_model.visual, 'proj') and self.clip_model.visual.proj is not None:
            embed_dim = self.clip_model.visual.proj.shape[1]
        else:
            # Model tipine göre ayarlayın. ViT-B/32 için genelde 512, ViT-L/14 için 768
            embed_dim = 512
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        image_features = self.clip_model.encode_image(x)
        logits = self.classifier(image_features)
        return logits

def get_model(num_classes, device):
    clip_model, _ = clip.load(CLIP_MODEL_NAME, device=device)
    model = CLIPClassifier(clip_model, num_classes).to(device)
    return model

# Modeli oluştur
model = get_model(num_classes, DEVICE)

# Eğitimde kullandığınız model dosyasıyla, aynı CLIP modeli seçili mi kontrol edin.
state_dict = torch.load(MODEL_PATH, map_location=DEVICE)
model.load_state_dict(state_dict)

model.eval()

test_dataset = TestDataset(test_df, TEST_IMG_DIR, transforms=get_test_transforms())
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

predictions = []
filenames = []

with torch.no_grad():
    for images, fnames in test_loader:
        images = images.to(DEVICE)
        outputs = model(images)
        preds = outputs.argmax(dim=1).cpu().numpy()
        for p, f in zip(preds, fnames):
            predictions.append(cities[p])
            filenames.append(f)

submission_df = test_df.copy()
submission_df['city'] = predictions
submission_df.to_csv("submission.csv", index=False)
print("Tahminler 'submission.csv' dosyasına kaydedildi.")


/tmp/ipykernel_23/2030481163.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(MODEL_PATH, map_location=DEVICE)


Tahminler 'submission.csv' dosyasına kaydedildi.
